<a href="https://colab.research.google.com/github/kiranbeethoju/Stock-Market-Forecast/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [283]:
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

start = '2004-02-01'
end = '2020-06-25'

tickers = ['BHARTIARTL.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(names)


                   ticker        Open   Adj Close
Date                                             
2004-02-03  BHARTIARTL.NS   62.298401   55.406151
2004-02-04  BHARTIARTL.NS   58.993000   56.486176
2004-02-05  BHARTIARTL.NS   60.576801   55.449345
2004-02-06  BHARTIARTL.NS   59.865200   60.395962
2004-02-09  BHARTIARTL.NS   64.731598   60.007133
...                   ...         ...         ...
2020-06-22  BHARTIARTL.NS  575.799988  581.049988
2020-06-23  BHARTIARTL.NS  581.049988  578.799988
2020-06-24  BHARTIARTL.NS  581.000000  557.349976
2020-06-25  BHARTIARTL.NS  553.900024  559.000000
2020-06-26  BHARTIARTL.NS  560.000000  560.400024

[4039 rows x 3 columns]


In [284]:
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]

In [285]:
close.describe()

,y
count,4039.000000
mean,289.853174
std,97.660760
min,54.498901
25%,263.781647
50%,301.534058
75%,346.222427
max,598.799988


In [286]:
!pip install fbprophet

In [294]:
from fbprophet import Prophet
from plotly.graph_objs import *

df = open

def run(df):
  df = df[4000:]
  #df = df[df['ds'].dt.dayofweek < 5]
  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=400)
  future.tail()
  forecast = m.predict(future)
  future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})
  mixed = pd.merge(df,future,on="ds",how="inner")
  mixed.tail(30)
  avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3
  mixed["avg_val"] = ''
  for i in range(0, len(mixed)):
    avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
    mixed['avg_val'].iloc[i] = avg
  mixed = mixed
  import plotly as py

  import plotly.graph_objects as go
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                      mode='lines+markers',
                      name='Predicted Opening Values'))
  fig.show()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                      mode='markers',
                      name='Predicted Opening Avg Values '))

  fig.show()
  return mixed

In [295]:
a = run(open)
b = run(close)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [296]:
a = a.rename(columns={"avg_val":"open_avg","y":"actual_open","yhat_upper_future":"yhat_upper_future_open","yhat_future":"yhat_future_open","yhat_lower_future":"yhat_lower_future_open"})
b = b.rename(columns={"avg_val":"open_avg","y":"actual_open","yhat_upper_future":"yhat_upper_future_close","yhat_future":"yhat_future_close","yhat_lower_future":"yhat_lower_future_close"})


In [297]:
c = pd.merge(a,b,on="ds",how="inner")

In [298]:
c

,ds,actual_open_x,yhat_future_open,yhat_lower_future_open,yhat_upper_future_open,open_avg_x,actual_open_y,yhat_future_close,yhat_lower_future_close,yhat_upper_future_close,open_avg_y
0,2020-05-04,502.450012,523.990473,506.112534,542.064366,524.056,532.500000,529.147935,511.479486,546.394936,529.007
1,2020-05-05,538.000000,531.392060,513.543326,549.915115,531.617,528.049988,536.535279,518.640910,553.424419,536.2
2,2020-05-06,531.750000,539.196597,521.542306,556.310426,539.016,546.000000,538.421259,520.938659,555.968079,538.443
3,2020-05-07,545.349976,535.702291,519.689394,553.904889,536.432,528.049988,536.264881,517.042834,553.341516,535.55
4,2020-05-08,540.000000,536.371158,518.901978,553.260381,536.178,530.049988,542.799588,525.931196,558.959270,542.563
5,2020-05-11,533.049988,540.643502,522.988557,559.489718,541.041,536.200012,544.542169,526.539359,561.675368,544.252
6,2020-05-12,533.000000,548.045090,530.129753,566.345173,548.173,559.849976,551.929513,536.613893,570.298017,552.947
7,2020-05-13,568.400024,555.849627,538.469317,575.234241,556.518,555.950012,553.815493,536.790446,571.024628,553.877
8,2020-05-14,551.099976,552.354709,535.362500,570.213440,552.644,539.849976,551.659115,534.061568,568.365935,551.362
9,2020-05-15,543.500000,553.022963,535.139218,570.978235,553.047,554.099976,558.193822,540.238125,575.573683,558.002
